In [1]:
import tarfile
import xml.dom.minidom
import os
import re
import pandas as pd
import numpy as np

In [2]:
sirens = np.load('/project/0_cleaning/output_cleaning/sirens.npy').tolist()
print(len(sirens))
print(type(sirens[0]))

3573
<class 'str'>


# Extract the XML files to the working directory

In [3]:
with tarfile.open('/var/www/bodacc-all.tar.gz', 'r:gz') as tar:
    tar.extractall()
with tarfile.open('/var/www/bodacc-2019.tar.gz', 'r:gz') as tar:
    tar.extractall()
with tarfile.open('/var/www/bodacc-2016.tar.gz', 'r:gz') as tar:
    tar.extractall()

# Parse judgment files

In [4]:
path = '/nadia/bodacc/2017/'
for i, file in enumerate(os.listdir(path)):
    if i == 10:
        break
    if re.search('^PCL_BXA.+\.xml$', file):
        doc = xml.dom.minidom.parse(path + file)
        pretty_xml_as_string = doc.toprettyxml()
        print(pretty_xml_as_string)

<?xml version="1.0" ?>
<PCL_REDIFF xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="Bodacc_PCL_Redif_V17.xsd">
	<parution>20170202</parution>
	<dateParution>2017-10-20</dateParution>
	<annonces>
		<annonce>
			<typeAnnonce>
				<creation/>
			</typeAnnonce>
			<nojo>000000040181183</nojo>
			<numeroAnnonce>1441</numeroAnnonce>
			<numeroDepartement>04</numeroDepartement>
			<tribunal>TRIBUNAL DE COMMERCE DE MANOSQUE</tribunal>
			<identifiantClient>040181183</identifiantClient>
			<personnePhysique>
				<nom>FILIPPA</nom>
				<prenom>Thierry</prenom>
			</personnePhysique>
			<nonInscrit>RCS non inscrit</nonInscrit>
			<adresse>
				<france>
					<numeroVoie>24</numeroVoie>
					<typeVoie>Cours</typeVoie>
					<nomVoie>Louis Pasteur</nomVoie>
					<complGeographique>&quot;Filippa Toituré&quot;</complGeographique>
					<codePostal>04130</codePostal>
					<ville>Volx</ville>
				</france>
			</adresse>
			<jugement>
				<famille>Jugement d'ouverture</f

In [3]:
years = ['2016', '2017', '2018', '2019']

rows = []
for year in years:
    
    path = f'/project/1_feature_extraction/extract_labels/{year}/'
    
    for i, file in enumerate(os.listdir(path)):
        #if i == 1000:
        #    break
        if re.search('^PCL_BXA.+\.xml$', file):
            
            doc = xml.dom.minidom.parse(path + file)
            advices = doc.getElementsByTagName('annonce')

            for advice in advices:
                # entry ID
                nojo = advice.getElementsByTagName('nojo')[0].firstChild.data

                # SIREN (if registered)
                try:
                    siren = advice.getElementsByTagName('numeroIdentificationRCS')[0].firstChild.data
                    siren = siren.replace(' ','')
                except:
                    siren = advice.getElementsByTagName('nonInscrit')[0].firstChild.data
                
                # Content (only if SIREN is of interest)
                if siren in sirens:

                    jugement_famille = advice.getElementsByTagName('famille')[0].firstChild.data
                    jugement_nature = advice.getElementsByTagName('nature')[0].firstChild.data
                    try:
                        jugement_date = advice.getElementsByTagName('date')[0].firstChild.data
                        jugement_text = advice.getElementsByTagName('complementJugement')[0].firstChild.data
                    except:
                        jugement_date = ''
                        jugement_text= ''                    

                    rows.append({
                        'nojo': nojo,
                        'siren': siren,
                        'jugement_famille': jugement_famille,
                        'jugement_nature': jugement_nature,
                        'jugement_date': jugement_date,
                        'jugement_text': jugement_text
                    })
                    
    print(f'{year} processed.')

2016 processed.
2017 processed.
2018 processed.
2019 processed.


In [4]:
df = pd.DataFrame(rows)
df.to_csv('judgments_for_sirens.csv', index=False)
df.to_excel('judgments_for_sirens.xlsx', index=False)
df.head()

,jugement_date,jugement_famille,jugement_nature,jugement_text,nojo,siren
0,16 juin 2016,Avis de dépôt,Dépôt de l'état des créances,L'état des créances est déposé au greffe où to...,BXA16169002236O,529319550
1,2016-10-26,Extrait de jugement,Jugement modifiant le plan de redressement,Jugement du tribunal de commerce de Orléans en...,4502BP2008B0012,502202252
2,25 juin 2015,Avis de dépôt,Dépôt de l'état des créances Loi de 1985,L'état des créances est déposé au greffe où to...,BXA161800024168,311168306
3,15 février 2016,Extrait de jugement,Jugement modifiant le plan de sauvegarde,Jugement modifiant le plan de sauvegarde.,BXA16061000288Z,352256200
4,21 janvier 2016,Extrait de jugement,Jugement modifiant le plan de redressement,Jugement modifiant le plan de redressement.,BXA16040003463G,468500541


In [5]:
sirens_financial_distress = df['siren'].unique().tolist()
len(sirens_financial_distress)

90

In [6]:
labels_df = pd.DataFrame()
labels_df['siren'] = sirens
labels_df['label'] = [int(siren in sirens_financial_distress) for siren in sirens]
labels_df.to_csv('/project/1_feature_extraction/output_feature_extraction/labels.csv', index=False)
labels_df.shape

(3573, 2)

In [7]:
count_fin_distress = labels_df['label'].sum()
print(f'{count_fin_distress} sirens in financial distress')

90 sirens in financial distress


Wikipedia: En droit français, une procédure collective place sous contrôle judiciaire le fonctionnement d'une entreprise en difficulté. Elle rassemble tous les créanciers et les prive du droit d'agir individuellement. L'objectif de la procédure est d'abord de permettre à l'entreprise de survivre (afin de préserver l'emploi et d'éviter les faillites en chaîne, celles des fournisseurs notamment), en procédant à un échelonnement voire à un effacement de tout ou partie de ses dettes.

Based on the above, we concluded that any company appearing in the 'procédure collective' file should be considered not credit worthiness for classification purpose. Whether or not a company appears in the 'procédure collective' file thus becomes our label.